In [1]:
#Import necessary libraries
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar100
from keras import regularizers, optimizers
import numpy as np

D:\Applications\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

 69918720/169001437 [===========>..................] - ETA: 50s

KeyboardInterrupt: 

In [3]:
#Convert train and test images to float values with a range from 0 to 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [5]:
#Number of classes in CIFAR-10
num_classes = 10

In [6]:
#Change class labels to categorical values
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [7]:
#Base Resolution of 32 x 32
baseRes = 32


In [8]:
#Initiate a  sequential Keras model
model = Sequential()


In [9]:
#Create convolution layer 1 with 32 filters
model.add(Conv2D(baseRes,
                 (3,3),
                 padding='same', 
                 activation='relu',
                 input_shape=x_train.shape[1:]))

In [10]:
model.add(BatchNormalization())

In [11]:
#Create convolution layer 2 with 32 filters
model.add(Conv2D(baseRes,
                 (3,3),
                 padding='same',
                 activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4)))

In [12]:
model.add(BatchNormalization())

In [13]:
#Add a max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
#Add a dropout layer
model.add(Dropout(0.2))

In [15]:
#Create convolution layer 3 with 64 filters
model.add(Conv2D(2*baseRes,
                 (3,3),
                 padding='same',
                 activation='relu'))

In [16]:
model.add(BatchNormalization())

In [17]:
#Create convolution layer 4 with 64 filters
model.add(Conv2D(2*baseRes,
                 (3,3),
                 padding='same',
                 activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4)))

In [18]:
model.add(BatchNormalization())

In [19]:
#Add a max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [20]:
#Add a droupout layer
model.add(Dropout(0.3))

In [21]:
#Create convolution layer 5 with 128 filters
model.add(Conv2D(4*baseRes, 
                 (3,3), 
                 padding='same', 
                 activation='relu'))

In [22]:
model.add(BatchNormalization())

In [23]:
#Create convolution layer 6 with 128 filters
model.add(Conv2D(4*baseRes, 
                 (3,3), 
                 padding='same', 
                 activation='relu', 
                 kernel_regularizer=regularizers.l2(1e-4)))

In [24]:
model.add(BatchNormalization())

In [25]:
#Add a max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [26]:
#Add a dropout layer
model.add(Dropout(0.4))

In [27]:
#Add a layer to flatten the values
model.add(Flatten())

In [28]:
#Add a dense layer with softmax activation to get the output for the 10 classes
model.add(Dense(num_classes, activation='softmax'))

In [29]:
#Summarize model structure
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

#### Data augmentation
This step creates multiple copies of the same image that have the following operations performed on them
1. Shift horizontally
2. Shift vertically
3. Rotate
4. Invert horizontally

In [30]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
    )

#Data generator
datagen.fit(x_train)

In [31]:
#Importing tensorboard and configuring a directory
from keras.callbacks import TensorBoard
tensorboard_1 = TensorBoard(log_dir="D:\Data\Tensorboard\model_base_final2")

Instructions for updating:
Use the retry module or similar alternatives.


In [32]:
#training
batch_size = 64
epochs=25

#Define optimizer
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=4*epochs,
                    verbose=1,
                    validation_data=(x_test,y_test),
                    callbacks=[tensorboard_1])


Epoch 1/100
781/781 [==============================] - 92s 118ms/step - loss: 2.1108 - acc: 0.3471 - val_loss: 1.7137 - val_acc: 0.4412
Epoch 2/100
781/781 [==============================] - 90s 115ms/step - loss: 1.6090 - acc: 0.4805 - val_loss: 1.7705 - val_acc: 0.4938
Epoch 3/100
781/781 [==============================] - 89s 114ms/step - loss: 1.4239 - acc: 0.5462 - val_loss: 1.3850 - val_acc: 0.5704
Epoch 4/100
781/781 [==============================] - 89s 113ms/step - loss: 1.3563 - acc: 0.5811 - val_loss: 1.2503 - val_acc: 0.6117
Epoch 5/100
781/781 [==============================] - 89s 114ms/step - loss: 1.2793 - acc: 0.6063 - val_loss: 1.1209 - val_acc: 0.6322
Epoch 6/100
781/781 [==============================] - 89s 114ms/step - loss: 1.2097 - acc: 0.6261 - val_loss: 1.0243 - val_acc: 0.6529
Epoch 7/100
781/781 [==============================] - 88s 113ms/step - loss: 1.1650 - acc: 0.6434 - val_loss: 1.2209 - val_acc: 0.6296
Epoch 8/100
781/781 [===========================

In [33]:
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 6s 593us/step

Test result: 83.890 loss: 0.545
